In [59]:
import os
import sqlite3
import pandas as pd

# Run this to go up a level
#os.chdir(os.path.dirname(os.path.abspath('')))
os.chdir('/Users/nikkivanhandel/CS6400/mfgdb/CS6400-MfgDB')

In [60]:
conn = sqlite3.connect("mfgdb")

builds_df = pd.read_sql_query("SELECT * FROM Builds", conn)
builds_df.sort_values('BuildID', inplace=True)
builds_df = builds_df.drop(0).reset_index(drop=True)
builds_df = builds_df.rename(columns={'Build ID:': 'BuildID'})

print("Builds Data:")
display(builds_df.tail())

Builds Data:


,BuildID,Nickname,Operator,DatePrinted,Customer,BuildPlateType,BuildPlateID,FileLocation,ParameterFileName,Successful,...,EstimatedPowderNeeded,PreBuildNotes,PostBuildNotes,RecoaterType,RecyclingState,DosingBoost,GasFlowVoltage,BuildShiftX,BuildShiftY,PowderID
65,B092,Trial Layout,"Caroline, Brooks",2023-12-14,Saldana,Full,--,Z:\EOS_CPU\Builds\B090_TestLayout_Config,Custom,,...,75.0,,Loud scraping on first couple layers but then ...,Hard,1x sieved,3.0,2.5,0.0,0.0,0
66,B093,Template,Caroline,2024-01-10,Caroline,Full,1,Z:\EOS_CPU\Builds\B093_OldRecoatDamage,Example Link,,...,,,Loud scraping on first couple layers but then ...,,1x sieved,,2.5,0.0,0.0,0
67,B094,Heavy Recoat Damage,"Caroline, Brooks",2024-01-17,Saldana,Full,--,Z:\EOS_CPU\Builds\B094_heavy_recoat_damage,LANL40,,...,52.5,,img 1 is from 1/16 the last image. the third l...,Hard,1x sieved,3.0,2.5,0.0,0.0,0
68,B095,Control GG,"Ken, Brooks",2024-02-02,Ken/Stebner,Full,--,Z:\EOS_CPU\Builds\B095_Ken_grainGrowth_316l,LANL40,,...,20.0,1st Training session for Ken.,"Build failure, deleted failing part but only d...",Hard,1x sieved,3.0,2.5,0.0,0.0,0
69,B096,Control GG,"Ken, Brooks",2024-02-06,Ken/Stebner,Full,--,Z:\EOS_CPU\Builds\B096_Ken_grainGrowth_316l,LANL40,,...,20.0,,,Hard,1x sieved,3.0,2.5,0.0,0.0,0


In [61]:
# Sensor Data 
sensors_df = pd.read_sql_query("SELECT * FROM Sensors", conn)

print("Sensors:")
display(sensors_df.head())

Sensors:


,Serial,Name,Model,DataKind,Manufacturer,Resolution,Depth
0,1,cam1,PHX-200S-MC,optical image,Lucid Vision,5472x3648,12
1,2,cam2,PHX-032S-MC,optical image,Lucid Vision,2048x1536,12
2,3,laser power,daq-power,voltage,NIST,0.0001,32
3,4,laser x,daq-galvo-x,x position,NIST,0.0001,32
4,5,laser y,daq-galvo-y,y position,NIST,0.0001,32


In [62]:
# Sensor data database
sensor_data = pd.read_sql_query("SELECT * FROM SensorData", conn)
display(sensor_data.head())

print(sensor_data.dtypes)

,RecordID,BuildID,SensorSerial,Value,Date,Time
0,1,B058,1,cam1_img_2023-01-06T15-36-40.png,2023-01-06,15:36:40
1,2,B058,1,cam1_img_2023-01-06T15-36-47.png,2023-01-06,15:36:47
2,3,B058,1,cam1_img_2023-01-06T15-36-54.png,2023-01-06,15:36:54
3,4,B058,1,cam1_img_2023-01-06T15-41-50.png,2023-01-06,15:41:50
4,5,B058,1,cam1_img_2023-01-06T15-41-59.png,2023-01-06,15:41:59


RecordID         int64
BuildID         object
SensorSerial    object
Value           object
Date            object
Time            object
dtype: object


In [63]:
# Analysis Case: Find images which correspond to failed builds
query = """
       with Failed_Builds as (
              Select *
              From Builds 
              Where Successful = 'False'
              )
              Select S.RecordID,
                     S.BuildID, 
                     S.SensorSerial, 
                     F.DatePrinted
              From Failed_Builds F 
              Join SensorData S
              On F.BuildID = S.BuildID
       """
# Use case: 
failed_builds = pd.read_sql_query(query, conn)
failed_builds.BuildID.unique() 

array(['B076', 'B071', 'B072'], dtype=object)

In [64]:
# Double check that these are the same as what the equivalent logic in pandas would tell us ! 
success_buildids = builds_df[builds_df.Successful=='False'].BuildID.unique()
sensor_buildids = sensor_data.BuildID.unique()

list(set(success_buildids) & set(sensor_buildids))

['B076', 'B071', 'B072']

In [65]:
# Analysis Case: Find images taken between date - date 
query2 = """
      Select *
      From SensorData
      WHERE DATE(Date) BETWEEN '2023-01-01' AND '2023-01-15'
"""
# Use case: 

Images_bt_dates = pd.read_sql_query(query2, conn)
display(Images_bt_dates)

,RecordID,BuildID,SensorSerial,Value,Date,Time
0,1,B058,1,cam1_img_2023-01-06T15-36-40.png,2023-01-06,15:36:40
1,2,B058,1,cam1_img_2023-01-06T15-36-47.png,2023-01-06,15:36:47
2,3,B058,1,cam1_img_2023-01-06T15-36-54.png,2023-01-06,15:36:54
3,4,B058,1,cam1_img_2023-01-06T15-41-50.png,2023-01-06,15:41:50
4,5,B058,1,cam1_img_2023-01-06T15-41-59.png,2023-01-06,15:41:59
...,...,...,...,...,...,...
823,824,B058,2,cam2_img_2023-01-06T21-54-36.png,2023-01-06,21:54:36
824,825,B058,2,cam2_img_2023-01-06T21-54-48.png,2023-01-06,21:54:48
825,826,B058,2,cam2_img_2023-01-06T21-55-37.png,2023-01-06,21:55:37
826,827,B058,2,cam2_img_2023-01-06T21-55-48.png,2023-01-06,21:55:48


In [66]:
# Analysis how long a build took
query3 = """
     with Image_Datetime as (
            Select RecordID, 
                   BuildID, 
                   DATETIME(Date ||' '|| Time) as Date_Time
            From SensorData
            )
      Select BuildID,
             (julianday(max(Date_Time)) - julianday(min(Date_Time))) * 24 as Time_Spent_Hours
      From Image_Datetime
      Group By BuildID
      Order by Time_Spent_Hours DESC  
"""
# Use case: 

Build_Time_Spent = pd.read_sql_query(query3, conn)
display(Build_Time_Spent)


,BuildID,Time_Spent_Hours
0,B096,6740.029722
1,B102,595.728611
2,B063,526.220556
3,B065,476.557500
4,B076,390.815278
5,B118,361.143889
6,B116,184.360278
7,B101,166.626389
8,B099,145.891667
9,B072,144.048611


In [67]:
conn.close()